In [ ]:
!pip install -Uqq fastbook

In [ ]:
!pip install virtualenv
!virtualenv venv
!source venv/bin/activate
!pip install fastbook


In [ ]:
import fastbook
fastbook.setup_book()

In [ ]:
!pip install duckduckgo-images-api

In [ ]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
search_images_ddg

In [ ]:
face_problem_types = ['DARK CIRCLES ON FACE','DARK SPOTS ON FACE','EYE BAGS','DRY SKIN','OILY SKIN','FACE REDNESS',
          'DULL SKIN','ROUGH TEXTURE ON FACE','LARGE PORES ON FACE','UNDER-EYE WRINKLES','HORMONAL ACNE',
          'BLACKHEADS','SEBACEOUS FILAMENTS','FOREHEAD WRINKLES', 'ACNE']
path = Path('Skin_Problems')

In [ ]:
if not path.exists():
    path.mkdir()
    for face_problem_type in face_problem_types :
      dest = (path/face_problem_type)
      dest.mkdir(exist_ok = True)
      results = search_images_ddg(f'{face_problem_type} face_problem')
      download_images(dest, urls = results)

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

In [ ]:
!zip -r Skin_Problems.zip Skin_Problems/

In [ ]:
from google.colab import files
files.download('Skin_Problems.zip')

In [ ]:
face_problems = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.1, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = face_problems.dataloaders(path)

In [ ]:
face_problems = face_problems.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2))
dls = face_problems.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True)

In [ ]:
face_problems = face_problems.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms(),
)
dls = face_problems.dataloaders(path,bs=16)

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=error_rate)
learn.fine_tune(50)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(1, nrows=1)

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')